In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy.stats
import statsmodels.formula.api as smf
from sklearn import linear_model
from google.colab import files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
L2Mc = pd.read_csv('/content/drive/Shared drives/DSO 510 Final Project/L2M cleaned.csv')
L2Mc.rename(columns ={'Unnamed: 0':'call id'}, inplace = True)

In [ ]:
L2Mc.head(5)

,call id,period,committing,disadvantaged,decision,comments,game_details,game_date,away_team,home_team,...,type2,season,playoff,Favored Team,FT_is_ST,Num_SP_in_FT,non Favored Team,Num_SP_in_NFT,NFT_is_ST,ST_in_call
0,0,Q4,Jae Crowder,Harrison Barnes,INC,Crowder (BOS) moves his left pivot foot after ...,"Warriors @ Celtics (Mar 01, 2015)",1-Mar-15,GSW,BOS,...,TRAVELING,2015,False,BOS,0,0.0,GSW,1.0,1.0,1.0
1,1,Q4,Avery Bradley,NaN,INC,Bradley (BOS) steps out of bounds while pump f...,"Warriors @ Celtics (Mar 01, 2015)",1-Mar-15,GSW,BOS,...,OUT OF BOUNDS,2015,False,BOS,0,0.0,GSW,1.0,1.0,1.0
2,2,Q4,Dante Cunningham,Wilson Chandler,INC,"Cunningham (NOP) clamps Chandler's (DEN) arm, ...","Pelicans @ Nuggets (Mar 01, 2015)",1-Mar-15,NOP,DEN,...,LOOSE BALL,2015,False,NOP,0,0.0,DEN,0.0,0.0,0.0
3,3,Q5,LeBron James,James Harden,INC,Harden (HOU) walks into space occupied by Jame...,"Cavaliers @ Rockets (Mar 01, 2015)",1-Mar-15,CLE,HOU,...,DOUBLE PERSONAL,2015,False,CLE,1,2.0,HOU,0.0,0.0,1.0
4,4,Q5,James Harden,LeBron James,INC,As Harden (HOU) starts his drive to the basket...,"Cavaliers @ Rockets (Mar 01, 2015)",1-Mar-15,CLE,HOU,...,OFFENSIVE,2015,False,HOU,0,0.0,CLE,2.0,1.0,1.0


In [ ]:
L2Mc['call id'] = np.arange(L2Mc.shape[0])

In [ ]:
L2Mc.shape

(5320, 36)

## Prepare dataset for experiment to evaluate relationship between number of super players (x) and team is favored or not in wrong call (y)

In [ ]:
num_empty_rows = L2Mc.shape[0]*2
L2Mc_exp= pd.DataFrame(index=range(num_empty_rows))
L2Mc_exp['call id'] = np.repeat(np.arange(L2Mc.shape[0]), 2)

In [ ]:
# assign team name

L2Mc_exp.loc[L2Mc_exp.index % 2 == 0, 'Team'] = L2Mc['Favored Team'].values
L2Mc_exp.loc[L2Mc_exp.index % 2 != 0, 'Team'] = L2Mc['non Favored Team'].values

In [ ]:
# assign team is favored or not
L2Mc_exp.loc[L2Mc_exp.index % 2 == 0, 'Favored'] = 1
L2Mc_exp.loc[L2Mc_exp.index % 2 != 0, 'Favored'] = 0

In [ ]:
L2Mc_exp.loc[L2Mc_exp.index % 2 == 0, 'Num of SP'] = L2Mc['Num_SP_in_FT'].values
L2Mc_exp.loc[L2Mc_exp.index % 2 != 0, 'Num of SP'] = L2Mc['Num_SP_in_NFT'].values

In [ ]:
df= pd.DataFrame(index=range(num_empty_rows))
df['call id'] = np.repeat(np.arange(L2Mc.shape[0]), 2)

In [ ]:
df = L2Mc_exp.merge(L2Mc, left_on=['call id'], right_on=['call id'], how='left')
df.head(5)

,call id,Team,Favored,Num of SP,period,committing,disadvantaged,decision,comments,game_details,...,type2,season,playoff,Favored Team,FT_is_ST,Num_SP_in_FT,non Favored Team,Num_SP_in_NFT,NFT_is_ST,ST_in_call
0,0,BOS,1.0,0.0,Q4,Jae Crowder,Harrison Barnes,INC,Crowder (BOS) moves his left pivot foot after ...,"Warriors @ Celtics (Mar 01, 2015)",...,TRAVELING,2015,False,BOS,0,0.0,GSW,1.0,1.0,1.0
1,0,GSW,0.0,1.0,Q4,Jae Crowder,Harrison Barnes,INC,Crowder (BOS) moves his left pivot foot after ...,"Warriors @ Celtics (Mar 01, 2015)",...,TRAVELING,2015,False,BOS,0,0.0,GSW,1.0,1.0,1.0
2,1,BOS,1.0,0.0,Q4,Avery Bradley,NaN,INC,Bradley (BOS) steps out of bounds while pump f...,"Warriors @ Celtics (Mar 01, 2015)",...,OUT OF BOUNDS,2015,False,BOS,0,0.0,GSW,1.0,1.0,1.0
3,1,GSW,0.0,1.0,Q4,Avery Bradley,NaN,INC,Bradley (BOS) steps out of bounds while pump f...,"Warriors @ Celtics (Mar 01, 2015)",...,OUT OF BOUNDS,2015,False,BOS,0,0.0,GSW,1.0,1.0,1.0
4,2,NOP,1.0,0.0,Q4,Dante Cunningham,Wilson Chandler,INC,"Cunningham (NOP) clamps Chandler's (DEN) arm, ...","Pelicans @ Nuggets (Mar 01, 2015)",...,LOOSE BALL,2015,False,NOP,0,0.0,DEN,0.0,0.0,0.0


In [ ]:
L2Mc_exp = df[['call id','Team','Favored','Num of SP','home_team','playoff','decision', 'ST_in_call']]
L2Mc_exp.head(5)

,call id,Team,Favored,Num of SP,home_team,playoff,decision,ST_in_call
0,0,BOS,1.0,0.0,BOS,False,INC,1.0
1,0,GSW,0.0,1.0,BOS,False,INC,1.0
2,1,BOS,1.0,0.0,BOS,False,INC,1.0
3,1,GSW,0.0,1.0,BOS,False,INC,1.0
4,2,NOP,1.0,0.0,DEN,False,INC,0.0


In [ ]:
L2Mc_exp.loc[(L2Mc_exp['Team'] == L2Mc_exp['home_team']), 'home'] = 1
L2Mc_exp.loc[(L2Mc_exp['Team'] != L2Mc_exp['home_team']), 'home'] = 0
L2Mc_exp.drop(['home_team'], axis=1)

<ipython-input-51-d7e420f0098e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  L2Mc_exp.loc[(L2Mc_exp['Team'] == L2Mc_exp['home_team']), 'home'] = 1


,call id,Team,Favored,Num of SP,playoff,decision,ST_in_call,home
0,0,BOS,1.0,0.0,False,INC,1.0,1.0
1,0,GSW,0.0,1.0,False,INC,1.0,0.0
2,1,BOS,1.0,0.0,False,INC,1.0,1.0
3,1,GSW,0.0,1.0,False,INC,1.0,0.0
4,2,NOP,1.0,0.0,False,INC,0.0,0.0
...,...,...,...,...,...,...,...,...
10635,5317,DEN,0.0,0.0,True,INC,0.0,1.0
10636,5318,MIA,1.0,0.0,True,INC,0.0,0.0
10637,5318,DEN,0.0,0.0,True,INC,0.0,1.0
10638,5319,DEN,1.0,0.0,True,INC,0.0,1.0


In [ ]:
L2Mc_exp.loc[(L2Mc_exp['playoff'] == False), 'playoff'] = 0
L2Mc_exp.loc[(L2Mc_exp['playoff'] == True), 'playoff'] = 1

In [ ]:
L2Mc_exp.loc[(L2Mc_exp['decision'] == 'IC'), 'decision'] = 1
L2Mc_exp.loc[(L2Mc_exp['decision'] == 'INC'), 'decision'] = 0

In [ ]:
L2Mc_exp.rename(columns ={'Num of SP':'Num_of_SP'}, inplace = True)

<ipython-input-54-da8f8b8febf8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  L2Mc_exp.rename(columns ={'Num of SP':'Num_of_SP'}, inplace = True)


In [ ]:
L2Mc_exp.head(5)
L2Mc_exp.shape

(10640, 9)

## Experiment: Regular Regression

In [ ]:
L2Mc_exp_one = L2Mc_exp.loc[L2Mc_exp['ST_in_call'] == 1]
L2Mc_exp_one.head(5)

,call id,Team,Favored,Num_of_SP,home_team,playoff,decision,ST_in_call,home
0,0,BOS,1.0,0.0,BOS,0,0,1.0,1.0
1,0,GSW,0.0,1.0,BOS,0,0,1.0,0.0
2,1,BOS,1.0,0.0,BOS,0,0,1.0,1.0
3,1,GSW,0.0,1.0,BOS,0,0,1.0,0.0
6,3,CLE,1.0,2.0,HOU,0,0,1.0,0.0


In [ ]:
L2Mc_exp_one[['Favored','Num_of_SP','home','decision']].corr()

<ipython-input-42-7b1c70d58bb3>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  L2Mc_exp_one[['Favored','Num_of_SP','home','decision']].corr()


,Favored,Num_of_SP,home
Favored,1.000000,-0.026204,0.019074
Num_of_SP,-0.026204,1.000000,-0.064819
home,0.019074,-0.064819,1.000000


In [ ]:
model = smf.ols(formula = 'Favored ~ Num_of_SP + playoff + home', data = L2Mc_exp_one).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                Favored   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.211
Date:                Tue, 28 Nov 2023   Prob (F-statistic):              0.304
Time:                        21:32:15   Log-Likelihood:                -2661.8
No. Observations:                3670   AIC:                             5332.
Df Residuals:                    3666   BIC:                             5357.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.5020      0.014     35.480   

In [ ]:
model = smf.ols(formula = 'Favored ~ Num_of_SP', data = L2Mc_exp_one).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                Favored   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     2.520
Date:                Tue, 28 Nov 2023   Prob (F-statistic):              0.112
Time:                        21:29:11   Log-Likelihood:                -2662.4
No. Observations:                3670   AIC:                             5329.
Df Residuals:                    3668   BIC:                             5341.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.5112      0.011     47.078      0.0

## Prepare dataset for experiment to evaluate relationship between average attendence (x) and team is favored or not in wrong call (y)

In [ ]:
attendance = pd.read_csv('/content/drive/Shared drives/DSO 510 Final Project/Attendance.csv')

In [ ]:
L2Mc_exp2 = df[['call id','Team','Favored','season', 'Num of SP','home_team','playoff','decision']]
L2Mc_exp2.head(5)

,call id,Team,Favored,season,Num of SP,home_team,playoff,decision
0,0,BOS,1.0,2015,0.0,BOS,False,INC
1,0,GSW,0.0,2015,1.0,BOS,False,INC
2,1,BOS,1.0,2015,0.0,BOS,False,INC
3,1,GSW,0.0,2015,1.0,BOS,False,INC
4,2,NOP,1.0,2015,0.0,DEN,False,INC


In [ ]:
L2Mc_exp2.loc[(L2Mc_exp2['Team'] == L2Mc_exp2['home_team']), 'home'] = 1
L2Mc_exp2.loc[(L2Mc_exp2['Team'] != L2Mc_exp2['home_team']), 'home'] = 0
L2Mc_exp2.drop(['home_team'], axis=1)

<ipython-input-68-cda20f66afcb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  L2Mc_exp2.loc[(L2Mc_exp2['Team'] == L2Mc_exp2['home_team']), 'home'] = 1


,call id,Team,Favored,season,Num of SP,playoff,decision,home
0,0,BOS,1.0,2015,0.0,False,INC,1.0
1,0,GSW,0.0,2015,1.0,False,INC,0.0
2,1,BOS,1.0,2015,0.0,False,INC,1.0
3,1,GSW,0.0,2015,1.0,False,INC,0.0
4,2,NOP,1.0,2015,0.0,False,INC,0.0
...,...,...,...,...,...,...,...,...
10635,5317,DEN,0.0,2023,0.0,True,INC,1.0
10636,5318,MIA,1.0,2023,0.0,True,INC,0.0
10637,5318,DEN,0.0,2023,0.0,True,INC,1.0
10638,5319,DEN,1.0,2023,0.0,True,INC,1.0


In [ ]:
L2Mc_exp2.loc[(L2Mc_exp2['playoff'] == False), 'playoff'] = 0
L2Mc_exp2.loc[(L2Mc_exp2['playoff'] == True), 'playoff'] = 1

In [ ]:
L2Mc_exp2.loc[(L2Mc_exp2['decision'] == 'IC'), 'decision'] = 1
L2Mc_exp2.loc[(L2Mc_exp2['decision'] == 'INC'), 'decision'] = 0

In [ ]:
L2Mc_exp2.rename(columns ={'Num of SP':'Num_of_SP'}, inplace = True)

<ipython-input-71-9f0a5346920e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  L2Mc_exp2.rename(columns ={'Num of SP':'Num_of_SP'}, inplace = True)


In [ ]:
L2Mc_exp2 = L2Mc_exp2.merge(attendance, left_on=['Team','season'], right_on=['team','Season'], how='left')

In [ ]:
L2Mc_exp2.dropna(subset = ['attendance'], inplace = True)

In [ ]:
L2Mc_exp2.isnull().sum()

call id       0
Team          0
Favored       0
season        0
Num_of_SP     0
home_team     0
playoff       0
decision      0
home          0
team          0
Season        0
attendance    0
dtype: int64

In [ ]:
model_attendance1 = smf.ols(formula = 'Favored ~ attendance', data = L2Mc_exp2).fit()
print(model_attendance1.summary())

                            OLS Regression Results                            
Dep. Variable:                Favored   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     2.886
Date:                Tue, 28 Nov 2023   Prob (F-statistic):             0.0894
Time:                        21:29:12   Log-Likelihood:                -7021.3
No. Observations:                9676   AIC:                         1.405e+04
Df Residuals:                    9674   BIC:                         1.406e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.5903      0.053     11.055      0.0

## Appendix

## Regular Regression for only playoff games

In [ ]:
L2Mc_exp_playoff = L2Mc_exp.loc[L2Mc_exp['playoff'] == 1]

In [ ]:
model_two = smf.ols(formula = 'Favored ~  home', data = L2Mc_exp_playoff).fit()
print(model_two.summary())

                            OLS Regression Results                            
Dep. Variable:                Favored   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     15.71
Date:                Tue, 28 Nov 2023   Prob (F-statistic):           8.04e-05
Time:                        23:06:34   Log-Likelihood:                -581.54
No. Observations:                 812   AIC:                             1167.
Df Residuals:                     810   BIC:                             1176.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4310      0.025     17.516      0.0

## Experiement: Logistic Regression

In [ ]:
model = smf.logit(formula = 'Favored ~ Num_of_SP + playoff + decision + home', data = L2Mc_exp).fit()
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.693031
         Iterations 3
                           Logit Regression Results                           
Dep. Variable:                Favored   No. Observations:                10640
Model:                          Logit   Df Residuals:                    10635
Method:                           MLE   Df Model:                            4
Date:                Tue, 28 Nov 2023   Pseudo R-squ.:               0.0001676
Time:                        21:29:11   Log-Likelihood:                -7373.9
converged:                       True   LL-Null:                       -7375.1
Covariance Type:            nonrobust   LLR p-value:                    0.6497
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.0086      0.031     -0.281      0.779      -0.069       0.051
playoff[T.1]      0.

In [ ]:
model1 = smf.logit(formula = 'Favored ~ home', data = L2Mc_exp).fit()
print(model1.summary())

Optimization terminated successfully.
         Current function value: 0.693096
         Iterations 3
                           Logit Regression Results                           
Dep. Variable:                Favored   No. Observations:                10640
Model:                          Logit   Df Residuals:                    10638
Method:                           MLE   Df Model:                            1
Date:                Tue, 28 Nov 2023   Pseudo R-squ.:               7.432e-05
Time:                        21:29:11   Log-Likelihood:                -7374.5
converged:                       True   LL-Null:                       -7375.1
Covariance Type:            nonrobust   LLR p-value:                    0.2951
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0203      0.027     -0.740      0.459      -0.074       0.033
home           0.0406      0.

In [ ]:
L2Mc_exp_one = L2Mc_exp.loc[L2Mc_exp['ST_in_call'] == 1]

In [ ]:
model_one = smf.logit(formula = 'Favored ~ Num_of_SP + playoff + decision + home', data = L2Mc_exp_one).fit()
print(model_one.summary())

Optimization terminated successfully.
         Current function value: 0.692652
         Iterations 3
                           Logit Regression Results                           
Dep. Variable:                Favored   No. Observations:                 3670
Model:                          Logit   Df Residuals:                     3665
Method:                           MLE   Df Model:                            4
Date:                Tue, 28 Nov 2023   Pseudo R-squ.:               0.0007144
Time:                        21:29:12   Log-Likelihood:                -2542.0
converged:                       True   LL-Null:                       -2543.9
Covariance Type:            nonrobust   LLR p-value:                    0.4577
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.0078      0.058      0.135      0.892      -0.105       0.121
playoff[T.1]      0.